# Estudo de Caso

## importação de bibliotecas e dados

In [ ]:
#instalação de biblioteca (opcional)
import pip
pip.main(["install","pandas","folium","matplotlib","geopy","requests","urllib"])

In [1]:
#importação de biblioteca
import folium 
import warnings
import urllib.request
import json
import time
import requests
import geopy.distance
import pandas as pd
import matplotlib.pyplot as plt
from email import header
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from numpy import dtype

In [2]:
#configurar pacotes
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#importação de dados
df_original = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/DADOS_SISREG.csv', dtype=str)
df_cbo = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CBO.csv', encoding='latin')
df_gravidade = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_GRAVIDADE.csv')
df_raca = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_RAÇA.csv')
df_sexo = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_SEXO.csv')
df_deslocamento = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_DESLOCAMENTO.csv')
df_cep = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/TABELA_CEP_BR.txt', dtype=str, sep="	", header=None)
df_cep.columns = ['CEP_USUARIO','UF','CIDADE','BAIRRO','LOGRADOURO']
df_cnes = pd.read_csv('../SISREG_Estudo_caso-AI_FALTAS_USUARIOS/dados/cnes_estabelecimentos.csv', encoding='latin', sep=";", dtype=str)


## Tratamento com os CEP's

In [4]:
#quantidade de CEP's importado
df_cep.shape

(849343, 5)

In [5]:
#criando um DataFrame somente os CEP's dos usuarios.
df_ceps_usuario = df_original['CEP_USUARIO']
df_ceps_usuario = pd.DataFrame(df_ceps_usuario)
df_ceps_usuario.drop_duplicates(inplace=True)
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO
0,73840000
2,89545000
3,06823250
4,06807540
5,89835000


In [6]:
#quantidade de CEP's localizado na base 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO    448
dtype: int64


In [7]:
#localizando endereço do usuario com a base do CEP's BR
df_ceps_usuario = pd.merge(df_ceps_usuario, df_cep, on=['CEP_USUARIO'], how='left')


In [8]:
#criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO']+ " - " + df_ceps_usuario['BAIRRO']+ " - " + df_ceps_usuario['CIDADE']+ " - " + df_ceps_usuario['UF']
df_ceps_usuario.dropna(inplace=True)
df_ceps_usuario.head(5)

,CEP_USUARIO,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO
2,06823250,SP,Embu das Artes,Jardim Santo Eduardo,Rua Panorama,Rua Panorama - Jardim Santo Eduardo - Embu das...
3,06807540,SP,Embu das Artes,Jardim Mimas,Rua Índia,Rua Índia - Jardim Mimas - Embu das Artes - SP
8,78559215,MT,Sinop,Residencial José Adriano Leitão,Rua Projetada 14,Rua Projetada 14 - Residencial José Adriano Le...
11,37503190,MG,Itajubá,Vila Poddis,Rua Marilda,Rua Marilda - Vila Poddis - Itajubá - MG
12,75907015,GO,Rio Verde,Vila Promissão,Rua 15,Rua 15 - Vila Promissão - Rio Verde - GO


In [9]:
#localizando lat e log 
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head(10)

,CEP_USUARIO,UF,CIDADE,BAIRRO,LOGRADOURO,END_COMPLETO,GEOLOCALIZADO
2,06823250,SP,Embu das Artes,Jardim Santo Eduardo,Rua Panorama,Rua Panorama - Jardim Santo Eduardo - Embu das...,"(Rua Panorama, Jardim Santo Eduardo, Jardim Do..."
3,06807540,SP,Embu das Artes,Jardim Mimas,Rua Índia,Rua Índia - Jardim Mimas - Embu das Artes - SP,None
8,78559215,MT,Sinop,Residencial José Adriano Leitão,Rua Projetada 14,Rua Projetada 14 - Residencial José Adriano Le...,None
11,37503190,MG,Itajubá,Vila Poddis,Rua Marilda,Rua Marilda - Vila Poddis - Itajubá - MG,"(Rua Marilda, Jardim Bernadete, Itajubá, Regiã..."
12,75907015,GO,Rio Verde,Vila Promissão,Rua 15,Rua 15 - Vila Promissão - Rio Verde - GO,"(Rua 15, Vila Promissão, Rio Verde, Região Geo..."
13,68500160,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,Travessa Mestre Olívio - Velha Marabá - Marabá...,"(Rua Mestre Olívio, Velha Marabá, Marabá, Regi..."
14,88520405,SC,Lages,Gethal,Rua Campos Salles,Rua Campos Salles - Gethal - Lages - SC,"(Rua Campos Sales, Gethal, Lages, Região Geogr..."
15,37505169,MG,Itajubá,Vila Rubens,Avenida Vinte e Um de Novembro,Avenida Vinte e Um de Novembro - Vila Rubens -...,"(Avenida Vinte e Um de Novembro, Vila Rubens, ..."
16,06872080,SP,Itapecerica da Serra,Palmeiras,Estrada dos Hengles,Estrada dos Hengles - Palmeiras - Itapecerica ...,None
21,06810095,SP,Embu das Artes,Jardim Flórida,Rua Flor do Campo,Rua Flor do Campo - Jardim Flórida - Embu das ...,None


In [10]:
#verificar LOCALIZAÇÃO lat e log não localizada (limpeza)
df_ceps_usuario.dropna(inplace=True)

In [11]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario.head(10)

,CEP_USUARIO,UF,CIDADE,BAIRRO,LOGRADOURO,LATITUDE_USUARIO,LONGITUDE_USUARIO
2,06823250,SP,Embu das Artes,Jardim Santo Eduardo,Rua Panorama,-23.671765,-46.796708
11,37503190,MG,Itajubá,Vila Poddis,Rua Marilda,-22.437679,-45.470801
12,75907015,GO,Rio Verde,Vila Promissão,Rua 15,-17.824684,-50.927737
13,68500160,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,-5.345212,-49.132309
14,88520405,SC,Lages,Gethal,Rua Campos Salles,-27.794880,-50.302253
15,37505169,MG,Itajubá,Vila Rubens,Avenida Vinte e Um de Novembro,-22.415951,-45.466432
22,58428370,PB,Campina Grande,Pedregal,Rua São Sebastião,-7.223133,-35.910324
25,68501030,PA,Marabá,Independência,Rua José Bonifácio,-5.396699,-49.135602
28,06823090,SP,Embu das Artes,Jardim Santo Eduardo,Rua Bristol,-23.669470,-46.798784
42,69088149,AM,Manaus,Jorge Teixeira,Rua Gergelim,-3.029542,-59.918626


In [12]:
#quantidade de CEP's localizado na base e tratado
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

Cep's unicos de usuarios:  CEP_USUARIO          142
UF                   142
CIDADE               142
BAIRRO               142
LOGRADOURO           142
LATITUDE_USUARIO     142
LONGITUDE_USUARIO    142
dtype: int64


In [13]:
df_original_c = pd.merge(df_original, df_ceps_usuario, how="inner", on="CEP_USUARIO")
df_original_c.head()

,CNS_USUARIO,NOME_USUARIO,IDADE_USUARIO,MUNICIPIO_USUARIO,IBGE_USUARIO,SEXO_USUARIO,RACA_COR_USUARIO,CEP_USUARIO,DESLOCAMENTO,CO_CNES,ESPECIALIDADE,GRAVIDADE,DATA_ATEND,TEMPO_ESPERA,CONF_ATEND,UF,CIDADE,BAIRRO,LOGRADOURO,LATITUDE_USUARIO,LONGITUDE_USUARIO
0,700000026383300,CICERA LEMOS DA SILVA,52,EMBU,351500,F,3,06823250,1,7805497,225270,3,2021-05-01,8,CONF,SP,Embu das Artes,Jardim Santo Eduardo,Rua Panorama,-23.671765,-46.796708
1,700000839746606,RONALDO PEREIRA,53,ITAJUBA,313240,M,1,37503190,1,6085210,225120,1,2021-05-01,9,CONF,MG,Itajubá,Vila Poddis,Rua Marilda,-22.437679,-45.470801
2,700001163333802,KAIO FIRMINO MARTINS,0,RIO VERDE,521880,M,3,75907015,1,0476447,225265,3,2021-05-01,17,CONF,GO,Rio Verde,Vila Promissão,Rua 15,-17.824684,-50.927737
3,700001195897809,WESLY COSTA PEREIRA,65,MARABA,150420,F,3,68500160,1,6378692,225112,3,2021-05-01,235,CONF,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,-5.345212,-49.132309
4,702205161788010,NAYARA CRISTINA NOGUEIRA NASCIMENTO,25,MARABA,150420,F,1,68500160,1,6378692,225112,3,2021-05-01,253,CONF,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,-5.345212,-49.132309


## Tratamento com os CNES

In [14]:
#quantidade de CNES's importado
df_cnes.shape

(380823, 16)

In [15]:
#criando um DataFrame somente os CEP's dos usuarios.
df_cnes_usuario = df_original_c['CO_CNES']
df_cnes_usuario = pd.DataFrame(df_cnes_usuario)
df_cnes_usuario.drop_duplicates(inplace=True)
df_cnes_usuario.dropna(inplace=True)
df_cnes_usuario.head(5)

,CO_CNES
0,7805497
1,6085210
2,0476447
3,6378692
5,2689812


In [16]:
#quantidade de CNES's localizado na base 
print("CNES's executante na base: ",df_cnes_usuario.count())

CNES's executante na base:  CO_CNES    20
dtype: int64


In [17]:
#localizando informações com a base do CNES
df_cnes_usuario = pd.merge(df_cnes_usuario,df_cnes, left_on='CO_CNES', right_on='CO_CNES', how='left')

In [19]:
#verificar CNES localizado (limpeza)
df_cnes_usuario.dropna(inplace=True)

In [20]:
#quantidade de CNES localizado na base e tratado
print("Cep's unicos de usuarios: ",df_cnes_usuario.count())

Cep's unicos de usuarios:  CO_CNES                 14
CO_UNIDADE              14
CO_UF                   14
CO_IBGE                 14
NU_CNPJ_MANTENEDORA     14
NO_RAZAO_SOCIAL         14
NO_FANTASIA             14
TP_GESTAO               14
CO_CEP                  14
NO_LOGRADOURO           14
NU_ENDERECO             14
NO_BAIRRO               14
NU_TELEFONE             14
NU_LATITUDE             14
NU_LONGITUDE            14
DS_TURNO_ATENDIMENTO    14
dtype: int64


In [21]:
df_cnes_usuario.head(5)

,CO_CNES,CO_UNIDADE,CO_UF,CO_IBGE,NU_CNPJ_MANTENEDORA,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_GESTAO,CO_CEP,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,NU_TELEFONE,NU_LATITUDE,NU_LONGITUDE,DS_TURNO_ATENDIMENTO
3,6378692,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
4,2689812,"4,20415E+12",42,420415,"7,84933E+13",PREFEITURA MUNICIPAL DE CELSO RAMOS,UNIDADE SANITARIA CENTRAL DE CELSO RAMOS,M,88598000,RUA JOSE MARTINELLI,198,CENTRO,(49)5471111,-2.763.257.358,-51.334.798.336,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
6,3030644,"1,5053E+12",15,150530,"5,13108E+12",PREFEITURA MUNICIPAL DE ORIXIMINA,UNIDADE BASICA DE SAUDE DR LAURO CORREA PINTO,M,68270000,RUA LAURO SODRE,2506,PERPETUO SOCORRO,93 3544-2136,-1.765.705,-55.860.205,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
8,4014138,"2,4112E+12",24,241120,"8,35889E+12",PREFEITURA MUNICIPAL DE SANTA CRUZ,HOSPITAL MUNICIPAL ALUIZIO BEZERRA,M,59200000,RUA SAO NICOLAU,35,CONJUNTO CONEGO MONT,(84)32917030,-62.293.204.456,-360.065.102.577,ATENDIMENTO CONTINUO DE 24 HORAS/DIA (PLANTAO:...
9,6670709,"1,10021E+12",11,110020,"5,90313E+12",PREFEITURA DO MUNICIPIO DE PORTO VELHO,CENTRO DE SAUDE MANOEL AMORIM DE MATOS,M,76801000,RUA ANGICO,5030,COHAB FLORESTA,69 39012805,-87.940.631,-638.873.164,"ATENDIMENTO NOS TURNOS DA MANHA, TARDE E NOITE"


In [22]:
df_original_cc = pd.merge(df_original_c, df_cnes_usuario, how="inner", on="CO_CNES")
df_original_cc.head()

,CNS_USUARIO,NOME_USUARIO,IDADE_USUARIO,MUNICIPIO_USUARIO,IBGE_USUARIO,SEXO_USUARIO,RACA_COR_USUARIO,CEP_USUARIO,DESLOCAMENTO,CO_CNES,ESPECIALIDADE,GRAVIDADE,DATA_ATEND,TEMPO_ESPERA,CONF_ATEND,UF,CIDADE,BAIRRO,LOGRADOURO,LATITUDE_USUARIO,LONGITUDE_USUARIO,CO_UNIDADE,CO_UF,CO_IBGE,NU_CNPJ_MANTENEDORA,NO_RAZAO_SOCIAL,NO_FANTASIA,TP_GESTAO,CO_CEP,NO_LOGRADOURO,NU_ENDERECO,NO_BAIRRO,NU_TELEFONE,NU_LATITUDE,NU_LONGITUDE,DS_TURNO_ATENDIMENTO
0,700001195897809,WESLY COSTA PEREIRA,65,MARABA,150420,F,3,68500160,1,6378692,225112,3,2021-05-01,235,CONF,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,-5.345212,-49.132309,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
1,702205161788010,NAYARA CRISTINA NOGUEIRA NASCIMENTO,25,MARABA,150420,F,1,68500160,1,6378692,225112,3,2021-05-01,253,CONF,PA,Marabá,Velha Marabá,Travessa Mestre Olívio,-5.345212,-49.132309,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
2,700002990741210,SIVANILDO DE CARVALHO,35,MARABA,150420,M,4,68501030,1,6378692,225112,3,2021-05-01,240,CONF,PA,Marabá,Independência,Rua José Bonifácio,-5.396699,-49.135602,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
3,700505503044258,JULIANA DEYD TRINDADE BARROS,28,MARABA,150420,F,4,68500140,1,6378692,225112,3,2021-05-01,235,CONF,PA,Marabá,Velha Marabá,Avenida Marabá,-5.348179,-49.127620,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE
4,700600443219663,CARLAN CARVALHO FERNADES,39,MARABA,150420,M,4,68501340,1,6378692,225112,3,2021-05-01,233,CONF,PA,Marabá,Liberdade,Rua Ademir Martins,-5.387073,-49.131679,"1,50421E+12",15,150420,"1,05346E+13",NEUROCARE SERVICOS NEUROLOGICOS LTDA,NEUROCARE,M,68509060,SETE,S/N,NOVA MARABA,94991933743,-534.617.207,-491.044.235.229,ATENDIMENTOS NOS TURNOS DA MANHA E A TARDE


## Calculando distancia (usuario x unidade executante)

In [23]:
#tratamento nos dados
df_original_cc = df_original_cc.astype({
    'LATITUDE_USUARIO':float,
    'LONGITUDE_USUARIO':float,
    'NU_LONGITUDE':float,
    'NU_LATITUDE':float,
})


ValueError: could not convert string to float: '-534.617.207'

In [ ]:
#verificar dados (limpeza)
df_original_cc.dropna(inplace=True)
df_original_cc.info()

In [ ]:
df_original_cc.head(10)

In [ ]:
#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_original_cc.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.NU_LATITUDE, row.NU_LONGITUDE)

    df_original_cc.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeEXE).km, 2)

#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_tratado.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.LATITUDE_UNIDADEEXE, row.LONGITUDE_UNIDADEEXE

    df_tratado.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeSOL).km, 2)

In [ ]:
df_original_cc.head(5)

In [ ]:
#quantidade de CNES's localizado na base 
df_cnes_usuario.info()

In [ ]:
df_cnes_usuario.head(5)

In [ ]:
df_cnes = df_cnes.drop([])

In [ ]:
df_cnes.head(10)

In [ ]:
df_cnes.info()

In [ ]:
#Consultar endereço com a API (CEP) (opcional)
for idx, row in df_ceps_usuario.iterrows():
    
    cep = row.CEP_USUARIO
    url = 'https://viacep.com.br/ws/%s/json/' % cep
    headers = {'User-Agent': 'Autociencia/1.0'}
    requisicao = urllib.request.Request(url, headers=headers, method='GET')
    usuario = urllib.request.urlopen(requisicao)
    conteudo = usuario.read().decode('utf-8')
    endereco = json.loads(conteudo)
    #time.sleep(1)
    usuario.close()    
    print(cep)

    if "cep" in endereco:
        df_ceps_usuario.at[idx, 'LOGRADOURO_USUARIO'] = endereco ['logradouro']
        df_ceps_usuario.at[idx, 'BAIRRO_USUARIO'] = endereco ['bairro']
        df_ceps_usuario.at[idx, 'CIDADE_USUARIO'] = endereco ['localidade']
        df_ceps_usuario.at[idx, 'ESTADO_USUARIO'] = endereco ['uf']

    else:
        
        df_ceps_usuario.at[idx, 'LOGRADOURO_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'BAIRRO_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'CIDADE_USUARIO'] = 'Não Localizado'
        df_ceps_usuario.at[idx, 'ESTADO_USUARIO'] = 'Não Localizado'

In [ ]:
df_ceps_usuario.head(5)

In [ ]:
#informações dos dados original
df_original.info()

In [ ]:
#quantidade de CEP's para localizar 
print("Cep's unicos de usuarios: ",df_ceps_usuario.count())

In [ ]:
#Criando um endereço completo para realizar pesquisa de lat e log
df_ceps_usuario ["END_COMPLETO"] = df_ceps_usuario['LOGRADOURO_USUARIO']+ "-" + df_ceps_usuario['BAIRRO_USUARIO'] + "-" +df_ceps_usuario['CIDADE_USUARIO']+ "-" +df_ceps_usuario['ESTADO_USUARIO']
df_ceps_usuario.head()

In [ ]:
#localizando lat e log 
locator = Nominatim(user_agent='localizacao')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_ceps_usuario["GEOLOCALIZADO"] = df_ceps_usuario["END_COMPLETO"].apply(geocode)
df_ceps_usuario.head()

In [ ]:
#verificar LOCALIZAÇÃO lat e log (limpeza)
df_ceps_usuario.dropna(inplace=True)

In [ ]:
#colocando a LATITUDE e LONGITUDE na tabela
df_ceps_usuario['LATITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.latitude)
df_ceps_usuario['LONGITUDE_USUARIO'] = df_ceps_usuario['GEOLOCALIZADO'].apply(lambda loc: loc.longitude)
df_ceps_usuario.drop('END_COMPLETO', axis=1, inplace=True)
df_ceps_usuario.drop('GEOLOCALIZADO', axis=1, inplace=True)
df_ceps_usuario

In [ ]:
df_tratado = pd.merge(df_original, df_ceps_usuario, how="inner", on="CEP_USUARIO")
df_tratado.head()

In [ ]:
#calcular a distancia da unidade executante e USUARIO (LAT E LON)
for idx, row in df_tratado.iterrows():
    coord_usuario = (row.LATITUDE_USUARIO, row.LONGITUDE_USUARIO)
    coord_unidadeEXE = (row.LATITUDE_UNIDADEEXE, row.LONGITUDE_UNIDADEEXE

    df_tratado.at[idx, "DISTANCIA_KM_USUARIO_UNIDADESOL"] = round(geopy.distance.distance(coord_usuario,coord_unidadeSOL).km, 2)

In [ ]:
df_tratado.head(10)

In [ ]:
df_tratado = pd.merge(df_original, df_ceps_usuario, how="inner", on="CEP_USUARIO")
df_tratado.head()